# Production Ingestion Pipeline - Abu Dhabi Government Knowledge Base

## Overview
This notebook serves as the **source of truth** for all production ingestion and tuning operations for the Abu Dhabi Government Knowledge Base (WoG) system.

### Key Features
- **🗄️ Production Database**: Creates and manages `WoG_Prod` database
- **📊 Contextual RAG**: Advanced section-to-chunk mapping with 105 sections per document
- **🌐 Remote Docling**: GPU-accelerated document processing with rich section extraction
- **🔍 Language Filtering**: Processes only 'en' and 'ar+en' documents
- **📋 Hierarchy Management**: Handles null values in government_entity, functional_domain, document_category
- **⚡ Performance Monitoring**: Step-by-step metrics and optimization

### Architecture
- **Database**: PostgreSQL + PGVector with 3072D embeddings
- **Processing**: Remote Docling server at `http://74.162.37.71:5001/v1alpha/convert/file`
- **Chunking**: 250-token chunks with 30-token overlap using tiktoken
- **Embedding**: text-embedding-3-large with token-aware generation
- **Search**: Cosine similarity with metadata filtering

---

## Step 1: Environment Setup and Configuration

### Initialize Environment Variables and Dependencies

In [1]:
import os
import sys
import time
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import psycopg2
from datetime import datetime

# Load environment variables
load_dotenv()

# Add src directory to path
sys.path.append(str(Path.cwd() / 'src'))

# Configuration
PRODUCTION_DATABASE_NAME = "WoG_Prod"
PRODUCTION_KB_FOLDER = "knowledge_base_prod"
EMBEDDING_MODEL = "text-embedding-3-large"
VECTOR_DIMENSION = 3072
ALLOWED_LANGUAGES = ['en', 'ar+en']

print("🔧 Production Ingestion Pipeline Initialized")
print(f"📊 Database: {PRODUCTION_DATABASE_NAME}")
print(f"📁 Source Folder: {PRODUCTION_KB_FOLDER}")
print(f"🔤 Allowed Languages: {ALLOWED_LANGUAGES}")
print(f"🎯 Embedding Model: {EMBEDDING_MODEL} ({VECTOR_DIMENSION}D)")
print(f"🌐 Remote Docling: {os.getenv('DOCLING_SERVER_URL')}")
print(f"⏰ Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

🔧 Production Ingestion Pipeline Initialized
📊 Database: WoG_Prod
📁 Source Folder: knowledge_base_prod
🔤 Allowed Languages: ['en', 'ar+en']
🎯 Embedding Model: text-embedding-3-large (3072D)
🌐 Remote Docling: http://74.162.37.71:5001/v1alpha/convert/file
⏰ Started: 2025-07-17 00:54:14


## Step 2: Create Production Database

### Create WoG_Prod Database with PGVector Extension

In [2]:
def create_production_database():
    """Create WoG_Prod database with PGVector extension"""
    try:
        # Connect to default postgres database to create new database
        conn = psycopg2.connect(
            host="localhost",
            database="postgres",
            user=os.getenv('DB_USER', 'mustaqmollah'),
            password=os.getenv('DB_PASSWORD', '')
        )
        conn.autocommit = True
        
        with conn.cursor() as cur:
            # Check if database exists
            cur.execute(f"SELECT 1 FROM pg_catalog.pg_database WHERE datname = '{PRODUCTION_DATABASE_NAME}'")
            exists = cur.fetchone()
            
            if exists:
                print(f"⚠️  Database '{PRODUCTION_DATABASE_NAME}' already exists")
                response = input("Do you want to drop and recreate it? (y/N): ")
                if response.lower() == 'y':
                    cur.execute(f"DROP DATABASE IF EXISTS {PRODUCTION_DATABASE_NAME}")
                    print(f"🗑️  Dropped existing database '{PRODUCTION_DATABASE_NAME}'")
                else:
                    print(f"✅ Using existing database '{PRODUCTION_DATABASE_NAME}'")
                    conn.close()
                    return True
            
            # Create new database
            cur.execute(f"CREATE DATABASE {PRODUCTION_DATABASE_NAME}")
            print(f"🎉 Created database '{PRODUCTION_DATABASE_NAME}'")
        
        conn.close()
        
        # Connect to new database and enable PGVector
        prod_conn = psycopg2.connect(
            host="localhost",
            database=PRODUCTION_DATABASE_NAME,
            user=os.getenv('DB_USER', 'mustaqmollah'),
            password=os.getenv('DB_PASSWORD', '')
        )
        prod_conn.autocommit = True
        
        with prod_conn.cursor() as cur:
            # Enable PGVector extension
            cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
            print("✅ PGVector extension enabled")
            
            # Verify vector support
            cur.execute("SELECT 1")
            print(f"✅ Database connection verified: {PRODUCTION_DATABASE_NAME}")
        
        prod_conn.close()
        return True
        
    except Exception as e:
        print(f"❌ Failed to create production database: {e}")
        return False

# Create production database
db_success = create_production_database()
print(f"📊 Database Status: {'✅ Success' if db_success else '❌ Failed'}")

❌ Failed to create production database: database "wog_prod" already exists

📊 Database Status: ❌ Failed


## Step 3: Create Database Schema

### Create Knowledge Base Tables with Hierarchy Support

In [3]:
# Update environment to use production database
os.environ['KB_DATABASE_URL'] = f"postgresql://mustaqmollah@localhost:5432/{PRODUCTION_DATABASE_NAME}"

# Import database models and manager
from models import (
    get_create_entities_table_sql,
    get_create_domains_table_sql,
    get_create_categories_table_sql,
    get_create_knowledge_base_table_sql,
    get_create_kb_indexes_sql
)
from database import KnowledgeBaseManager

def create_database_schema():
    """Create all database tables and indexes"""
    try:
        # Initialize database manager
        kb_manager = KnowledgeBaseManager()
        
        print("🏗️  Creating database schema...")
        
        # Create tables
        tables = [
            ("entities", get_create_entities_table_sql()),
            ("functional_domains", get_create_domains_table_sql()),
            ("document_categories", get_create_categories_table_sql()),
            ("knowledge_base", get_create_knowledge_base_table_sql(EMBEDDING_MODEL))
        ]
        
        for table_name, sql in tables:
            try:
                with kb_manager.get_connection() as conn:
                    with conn.cursor() as cur:
                        cur.execute(sql)
                        conn.commit()
                print(f"✅ Created table: {table_name}")
            except Exception as e:
                print(f"❌ Failed to create table {table_name}: {e}")
                return False
        
        # Create indexes
        print("🔍 Creating indexes...")
        try:
            with kb_manager.get_connection() as conn:
                with conn.cursor() as cur:
                    cur.execute(get_create_kb_indexes_sql(EMBEDDING_MODEL))
                    conn.commit()
            print("✅ Created indexes")
        except Exception as e:
            print(f"❌ Failed to create indexes: {e}")
            return False
        
        # Grant permissions
        print("🔐 Granting permissions...")
        try:
            with kb_manager.get_connection() as conn:
                with conn.cursor() as cur:
                    tables = ['entities', 'functional_domains', 'document_categories', 'knowledge_base']
                    for table in tables:
                        cur.execute(f'GRANT ALL PRIVILEGES ON TABLE {table} TO PUBLIC')
                    conn.commit()
            print("✅ Permissions granted")
        except Exception as e:
            print(f"❌ Failed to grant permissions: {e}")
            return False
        
        return True
        
    except Exception as e:
        print(f"❌ Failed to create database schema: {e}")
        return False

# Create schema
schema_success = create_database_schema()
print(f"🏗️  Schema Status: {'✅ Success' if schema_success else '❌ Failed'}")

✅ Knowledge Base PostgreSQL connection pool initialized (max_size=10)
✅ PGVector extension enabled for Knowledge Base
🔄 Creating Knowledge Base hierarchy tables...
🔧 Granting PUBLIC access to all WoG tables...
🌱 Seeding Knowledge Base with Excel hierarchy data...
✅ Knowledge Base database initialized with hierarchy data
✅ Knowledge Base PostgreSQL connection pool initialized (max_size=10)
✅ PGVector extension enabled for Knowledge Base
🔄 Creating Knowledge Base hierarchy tables...
🔧 Granting PUBLIC access to all WoG tables...
🌱 Seeding Knowledge Base with Excel hierarchy data...
✅ Knowledge Base database initialized with hierarchy data
🏗️  Creating database schema...
✅ Created table: entities
✅ Created table: functional_domains
✅ Created table: document_categories
✅ Created table: knowledge_base
🔍 Creating indexes...
✅ Created indexes
🔐 Granting permissions...
✅ Permissions granted
🏗️  Schema Status: ✅ Success


## Step 4: Generate Production YAML Mapping

### Process Excel Hierarchy Data with Language Filtering

In [4]:
# Import mapping generator
sys.path.append(str(Path.cwd() / 'scripts'))
from generate_mapping import KBMappingGenerator

def generate_production_mapping():
    """Generate YAML mapping for production data"""
    try:
        print("📋 Generating production YAML mapping...")
        
        # Initialize mapping generator with production folder
        excel_file = "knowledge_base_hierarchy_data.xlsx"
        version = "1.2"
        
        generator = KBMappingGenerator(
            excel_file=excel_file, 
            version=version,
            knowledge_base_folder=PRODUCTION_KB_FOLDER
        )
        
        print(f"📁 Production folder: {generator.knowledge_base_dir}")
        print(f"📊 Excel file: {excel_file}")
        print(f"📝 Version: v{version}")
        
        # Check if production folder exists
        if not generator.knowledge_base_dir.exists():
            print(f"❌ Production folder does not exist: {generator.knowledge_base_dir}")
            print("Please create the knowledge_base_prod folder and add your production documents")
            return False
        
        # Generate mapping
        success = generator.generate_yaml_mapping()
        
        if success:
            print("✅ Production YAML mapping generated successfully")
            
            # Show mapping statistics
            mapping_file = generator.config_dir / f"mapping_v{version}.yaml"
            if mapping_file.exists():
                import yaml
                with open(mapping_file, 'r') as f:
                    mapping_data = yaml.safe_load(f)
                
                print(f"📊 Mapping Statistics:")
                print(f"   - Total documents: {mapping_data.get('total_documents', 0)}")
                print(f"   - Available documents: {mapping_data.get('available_documents', 0)}")
                print(f"   - Missing documents: {mapping_data.get('missing_documents', 0)}")
                print(f"   - Entities: {len(mapping_data.get('entities', []))}")
                print(f"   - Domains: {len(mapping_data.get('functional_domains', []))}")
                print(f"   - Categories: {len(mapping_data.get('document_categories', []))}")
                
                # Show language distribution
                languages = {}
                for doc in mapping_data.get('document_mappings', []):
                    lang = doc.get('language', 'unknown')
                    languages[lang] = languages.get(lang, 0) + 1
                
                print(f"   - Languages: {languages}")
                
                # Show available vs missing breakdown
                available_docs = [d for d in mapping_data.get('document_mappings', []) if d.get('status') == 'available']
                missing_docs = [d for d in mapping_data.get('document_mappings', []) if d.get('status') == 'missing']
                
                print(f"\n📄 Available Documents ({len(available_docs)}):")
                for doc in available_docs[:10]:  # Show first 10
                    print(f"   - {doc['excel_title']} ({doc['language']})")
                if len(available_docs) > 10:
                    print(f"   ... and {len(available_docs) - 10} more")
                
                print(f"\n❓ Missing Documents ({len(missing_docs)}):")
                for doc in missing_docs[:5]:  # Show first 5
                    print(f"   - {doc['excel_title']} ({doc['language']})")
                if len(missing_docs) > 5:
                    print(f"   ... and {len(missing_docs) - 5} more")
            
            return True
        else:
            print("❌ Failed to generate production YAML mapping")
            return False
            
    except Exception as e:
        print(f"❌ Error generating production mapping: {e}")
        return False

# Generate production mapping
mapping_success = generate_production_mapping()
print(f"📋 Mapping Status: {'✅ Success' if mapping_success else '❌ Failed'}")

📋 Generating production YAML mapping...
🔧 KB Mapping Generator initialized
📁 Knowledge Base dir: /Users/mustaqmollah/Desktop/GovGPT/ContextualRag/govgpt-kb/knowledge_base_prod
📝 Version: v1.2
📊 Using folder: knowledge_base_prod
📁 Production folder: /Users/mustaqmollah/Desktop/GovGPT/ContextualRag/govgpt-kb/knowledge_base_prod
📊 Excel file: knowledge_base_hierarchy_data.xlsx
📝 Version: v1.2
🔄 Generating YAML mapping v1.2...
📖 Reading Excel file: knowledge_base_hierarchy_data.xlsx
✅ Read 24 rows from Excel file
⏭️  Skipping document 'CX Effortless_Guide_AR' - language 'ar' not in allowed list ['en', 'ar+en']
⏭️  Skipping document 'Abu Dhabi Government Finance Policy Manual_AR' - language 'ar' not in allowed list ['en', 'ar+en']
⏭️  Skipping document 'HR - Government Employee Guide Version 1 - AR' - language 'ar' not in allowed list ['en', 'ar+en']
⏭️  Skipping document 'HR - Government Employee Guide Version 2 - AR' - language 'ar' not in allowed list ['en', 'ar+en']
⏭️  Skipping documen

## Step 5: Populate Hierarchy Tables

### Load Entities, Domains, and Categories from YAML

In [5]:
# Import database populator
from populate_database import KBDatabasePopulator

def populate_hierarchy_tables():
    """Populate hierarchy tables from YAML mapping"""
    try:
        print("📊 Populating hierarchy tables...")
        
        # Initialize database populator
        populator = KBDatabasePopulator(version="1.2")
        
        # Load production mapping
        mapping_file = Path.cwd() / "config" / "mapping_v1.2.yaml"
        if not mapping_file.exists():
            print(f"❌ Production mapping file not found: {mapping_file}")
            return False
        
        import yaml
        with open(mapping_file, 'r') as f:
            mapping_data = yaml.safe_load(f)
        
        # Populate hierarchy tables
        success = populator.populate_hierarchy_tables(mapping_data)
        
        if success:
            print("✅ Hierarchy tables populated successfully")
            
            # Verify table contents
            from database import get_kb_manager
            kb_manager = get_kb_manager()
            
            try:
                with kb_manager.get_connection() as conn:
                    with conn.cursor() as cur:
                        # Count entities
                        cur.execute("SELECT COUNT(*) FROM entities")
                        entity_count = cur.fetchone()[0]
                        
                        # Count domains
                        cur.execute("SELECT COUNT(*) FROM functional_domains")
                        domain_count = cur.fetchone()[0]
                        
                        # Count categories
                        cur.execute("SELECT COUNT(*) FROM document_categories")
                        category_count = cur.fetchone()[0]
                        
                        print(f"📊 Database Contents:")
                        print(f"   - Entities: {entity_count}")
                        print(f"   - Domains: {domain_count}")
                        print(f"   - Categories: {category_count}")
                        
                        # Show sample entities
                        cur.execute("SELECT name, display_name, code FROM entities LIMIT 5")
                        entities = cur.fetchall()
                        print(f"\n🏛️  Sample Entities:")
                        for entity in entities:
                            print(f"   - {entity[0]} ({entity[2]}) - {entity[1]}")
                        
                        # Show sample domains
                        cur.execute("SELECT name, display_name FROM functional_domains LIMIT 5")
                        domains = cur.fetchall()
                        print(f"\n🔧 Sample Domains:")
                        for domain in domains:
                            print(f"   - {domain[0]} - {domain[1]}")
                        
                        # Show sample categories
                        cur.execute("SELECT name, description FROM document_categories LIMIT 5")
                        categories = cur.fetchall()
                        print(f"\n📋 Sample Categories:")
                        for category in categories:
                            print(f"   - {category[0]} - {category[1]}")
                            
            except Exception as e:
                print(f"❌ Error verifying hierarchy tables: {e}")
                return False
            
            return True
        else:
            print("❌ Failed to populate hierarchy tables")
            return False
            
    except Exception as e:
        print(f"❌ Error populating hierarchy tables: {e}")
        return False

# Populate hierarchy tables
hierarchy_success = populate_hierarchy_tables()
print(f"📊 Hierarchy Status: {'✅ Success' if hierarchy_success else '❌ Failed'}")

✅ Knowledge Base PostgreSQL connection pool initialized (max_size=10)
✅ PGVector extension enabled for Knowledge Base
🔄 Creating Knowledge Base hierarchy tables...
🔧 Granting PUBLIC access to all WoG tables...
🌱 Seeding Knowledge Base with Excel hierarchy data...
✅ Knowledge Base database initialized with hierarchy data
📊 Populating hierarchy tables...
🔧 DocumentProcessor initialized
📁 Cache directory: cache
🚀 Processing method: Docling
🌐 Docling server URL: 'http://74.162.37.71:5001/v1alpha/convert/file'
💾 Cache enabled: True
🗂️  Use cache: True
🔧 DocumentProcessor initialized
📁 Cache directory: cache
🚀 Processing method: Docling
🌐 Docling server URL: 'http://74.162.37.71:5001/v1alpha/convert/file'
💾 Cache enabled: True
🗂️  Use cache: True
🧩 ChunkProcessor initialized
🧠 Embedding model: text-embedding-3-large
📐 Chunk size: 250, overlap: 30
⚡ Max workers: 8
💾 Cache directory: ./cache
📊 ProcessingLogger initialized for version 1.2
📁 Log file: logs/processing_v1.2.log
🔧 Knowledge Base po

## Step 6: Clear Cache for Fresh Processing

### Clear Document Processing Cache

In [6]:
import shutil

def clear_processing_cache():
    """Clear document processing cache for fresh processing"""
    try:
        cache_dir = Path.cwd() / "cache"
        
        if cache_dir.exists():
            # Count files before clearing
            cache_files = list(cache_dir.glob("*.json"))
            file_count = len(cache_files)
            
            print(f"🗑️  Clearing {file_count} cached files...")
            
            # Remove all cache files
            for cache_file in cache_files:
                cache_file.unlink()
                
            print(f"✅ Cache cleared: {file_count} files removed")
        else:
            print("📁 Cache directory does not exist - creating it")
            cache_dir.mkdir(exist_ok=True)
            
        return True
        
    except Exception as e:
        print(f"❌ Error clearing cache: {e}")
        return False

# Clear cache
cache_success = clear_processing_cache()
print(f"🗑️  Cache Status: {'✅ Cleared' if cache_success else '❌ Failed'}")

🗑️  Clearing 0 cached files...
✅ Cache cleared: 0 files removed
🗑️  Cache Status: ✅ Cleared


## Step 7: Process Production Documents

### Process Documents with Remote Docling and Generate Embeddings

In [7]:
def process_production_documents():
    """Process production documents with remote Docling"""
    try:
        print("🔄 Processing production documents...")
        
        # Initialize database populator
        populator = KBDatabasePopulator(version="1.2")
        
        # Load production mapping
        mapping_file = Path.cwd() / "config" / "mapping_v1.2.yaml"
        if not mapping_file.exists():
            print(f"❌ Production mapping file not found: {mapping_file}")
            return False
        
        import yaml
        with open(mapping_file, 'r') as f:
            mapping_data = yaml.safe_load(f)
        
        # Get available documents
        available_docs = [d for d in mapping_data.get('document_mappings', []) if d.get('status') == 'available']
        
        if not available_docs:
            print("⚠️  No available documents found for processing")
            return False
        
        print(f"📄 Found {len(available_docs)} available documents to process")
        
        # Process each document
        processed_count = 0
        failed_count = 0
        start_time = time.time()
        
        for i, doc_data in enumerate(available_docs, 1):
            print(f"\n📄 Processing document {i}/{len(available_docs)}: {doc_data['excel_title']}")
            print(f"   Language: {doc_data['language']}")
            print(f"   Entity: {doc_data['entity']}")
            print(f"   Domain: {doc_data['domain']}")
            print(f"   Category: {doc_data['category']}")
            
            # Create document entry
            from populate_database import DocumentEntry
            
            doc_entry = DocumentEntry(
                excel_title=doc_data['excel_title'],
                entity=doc_data['entity'],
                domain=doc_data['domain'],
                category=doc_data['category'],
                language=doc_data['language'],
                file_path=doc_data['file_path'],
                filename=doc_data['filename'],
                file_size=doc_data.get('file_size', 0),
                status='available'
            )
            
            # Process document
            doc_start_time = time.time()
            success = populator.populate_document(doc_entry)
            doc_end_time = time.time()
            
            if success:
                processed_count += 1
                print(f"   ✅ Success ({doc_end_time - doc_start_time:.2f}s)")
            else:
                failed_count += 1
                print(f"   ❌ Failed ({doc_end_time - doc_start_time:.2f}s)")
            
            # Show progress
            elapsed = time.time() - start_time
            avg_time = elapsed / i
            eta = avg_time * (len(available_docs) - i)
            
            print(f"   Progress: {i}/{len(available_docs)} ({i/len(available_docs)*100:.1f}%) - ETA: {eta/60:.1f}min")
        
        # Final summary
        total_time = time.time() - start_time
        print(f"\n📊 Processing Summary:")
        print(f"   - Total documents: {len(available_docs)}")
        print(f"   - Processed successfully: {processed_count}")
        print(f"   - Failed: {failed_count}")
        print(f"   - Success rate: {processed_count/len(available_docs)*100:.1f}%")
        print(f"   - Total time: {total_time/60:.1f} minutes")
        print(f"   - Average time per document: {total_time/len(available_docs):.1f} seconds")
        
        return processed_count > 0
        
    except Exception as e:
        print(f"❌ Error processing production documents: {e}")
        return False

# Process production documents
processing_success = process_production_documents()
print(f"🔄 Processing Status: {'✅ Success' if processing_success else '❌ Failed'}")

🔄 Processing production documents...
🔧 DocumentProcessor initialized
📁 Cache directory: cache
🚀 Processing method: Docling
🌐 Docling server URL: 'http://74.162.37.71:5001/v1alpha/convert/file'
💾 Cache enabled: True
🗂️  Use cache: True
🔧 DocumentProcessor initialized
📁 Cache directory: cache
🚀 Processing method: Docling
🌐 Docling server URL: 'http://74.162.37.71:5001/v1alpha/convert/file'
💾 Cache enabled: True
🗂️  Use cache: True
🧩 ChunkProcessor initialized
🧠 Embedding model: text-embedding-3-large
📐 Chunk size: 250, overlap: 30
⚡ Max workers: 8
💾 Cache directory: ./cache
📊 ProcessingLogger initialized for version 1.2
📁 Log file: logs/processing_v1.2.log
🔧 Knowledge Base populator initialized
📁 Mapping file: /Users/mustaqmollah/Desktop/GovGPT/ContextualRag/govgpt-kb/config/mapping_v1.2.yaml
📝 Version: v1.2
🧩 Using ChunkProcessor for chunking and embeddings
📄 Found 14 available documents to process

📄 Processing document 1/14: CX Abu Dhabi Government Tone of Voice Document
   Language: 

## Step 8: Verify Database Population

### Check Knowledge Base Table Contents and Performance

In [ ]:
def verify_database_population():
    """Verify database population and show statistics"""
    try:
        print("📊 Verifying database population...")
        
        # Initialize database manager
        kb_manager = KnowledgeBaseManager()
        
        with kb_manager.get_connection() as conn:
            with conn.cursor() as cur:
                # Get total chunk count
                cur.execute("SELECT COUNT(*) FROM knowledge_base")
                total_chunks = cur.fetchone()[0]
                
                # Get unique document count
                cur.execute("SELECT COUNT(DISTINCT document_name) FROM knowledge_base")
                unique_docs = cur.fetchone()[0]
                
                # Get embedding count
                cur.execute("SELECT COUNT(*) FROM knowledge_base WHERE embedding IS NOT NULL")
                embedding_count = cur.fetchone()[0]
                
                # Get language distribution
                cur.execute("""
                    SELECT 
                        COALESCE(entity, 'Unknown') as entity,
                        COUNT(*) as chunk_count,
                        COUNT(DISTINCT document_name) as doc_count
                    FROM knowledge_base 
                    GROUP BY entity 
                    ORDER BY chunk_count DESC
                """)
                entity_stats = cur.fetchall()
                
                # Get domain distribution
                cur.execute("""
                    SELECT 
                        COALESCE(domain, 'Unknown') as domain,
                        COUNT(*) as chunk_count,
                        COUNT(DISTINCT document_name) as doc_count
                    FROM knowledge_base 
                    GROUP BY domain 
                    ORDER BY chunk_count DESC
                """)
                domain_stats = cur.fetchall()
                
                # Get category distribution
                cur.execute("""
                    SELECT 
                        COALESCE(category, 'Unknown') as category,
                        COUNT(*) as chunk_count,
                        COUNT(DISTINCT document_name) as doc_count
                    FROM knowledge_base 
                    GROUP BY category 
                    ORDER BY chunk_count DESC
                """)
                category_stats = cur.fetchall()
                
                # Get sample documents
                cur.execute("""
                    SELECT 
                        document_name,
                        document_title,
                        entity,
                        domain,
                        category,
                        COUNT(*) as chunk_count
                    FROM knowledge_base 
                    GROUP BY document_name, document_title, entity, domain, category
                    ORDER BY chunk_count DESC
                    LIMIT 10
                """)
                sample_docs = cur.fetchall()
                
                # Show statistics
                print(f"📊 Database Statistics:")
                print(f"   - Total chunks: {total_chunks:,}")
                print(f"   - Unique documents: {unique_docs}")
                print(f"   - Chunks with embeddings: {embedding_count:,}")
                print(f"   - Embedding coverage: {embedding_count/total_chunks*100:.1f}%")
                print(f"   - Average chunks per document: {total_chunks/unique_docs:.1f}")
                
                print(f"\n🏛️  Entity Distribution:")
                for entity, chunk_count, doc_count in entity_stats:
                    print(f"   - {entity}: {chunk_count:,} chunks ({doc_count} docs)")
                
                print(f"\n🔧 Domain Distribution:")
                for domain, chunk_count, doc_count in domain_stats:
                    print(f"   - {domain}: {chunk_count:,} chunks ({doc_count} docs)")
                
                print(f"\n📋 Category Distribution:")
                for category, chunk_count, doc_count in category_stats:
                    print(f"   - {category}: {chunk_count:,} chunks ({doc_count} docs)")
                
                print(f"\n📄 Sample Documents:")
                for doc_name, doc_title, entity, domain, category, chunk_count in sample_docs:
                    print(f"   - {doc_name} ({chunk_count} chunks)")
                    print(f"     Title: {doc_title}")
                    print(f"     Hierarchy: {entity} > {domain} > {category}")
                
                # Test vector search performance
                print(f"\n🔍 Testing Vector Search Performance...")
                
                # Generate a test embedding
                import numpy as np
                test_embedding = np.random.random(VECTOR_DIMENSION).tolist()
                
                # Test search speed
                search_start = time.time()
                results = kb_manager.search_kb_similar_vectors(test_embedding, limit=10)
                search_time = time.time() - search_start
                
                print(f"   - Search time: {search_time*1000:.1f}ms")
                print(f"   - Results found: {len(results)}")
                print(f"   - Performance: {'✅ Good' if search_time < 1.0 else '⚠️ Slow'}")
                
                return True
                
    except Exception as e:
        print(f"❌ Error verifying database population: {e}")
        return False

# Verify database population
verification_success = verify_database_population()
print(f"📊 Verification Status: {'✅ Success' if verification_success else '❌ Failed'}")

📊 Verifying database population...
✅ Knowledge Base PostgreSQL connection pool initialized (max_size=10)
✅ PGVector extension enabled for Knowledge Base
🔄 Creating Knowledge Base hierarchy tables...
🔧 Granting PUBLIC access to all WoG tables...
🌱 Seeding Knowledge Base with Excel hierarchy data...
✅ Knowledge Base database initialized with hierarchy data
📊 Database Statistics:
   - Total chunks: 1,717
   - Unique documents: 15
   - Chunks with embeddings: 1,717
   - Embedding coverage: 100.0%
   - Average chunks per document: 114.5

🏛️  Entity Distribution:
   - DGE: 1,296 chunks (10 docs)
   - Human Resource Authority: 256 chunks (3 docs)
   - Department of Finance: 129 chunks (1 docs)
   - Unknown: 36 chunks (1 docs)

🔧 Domain Distribution:
   - IT: 518 chunks (1 docs)
   - Procurement: 453 chunks (6 docs)
   - CX: 361 chunks (4 docs)
   - HR: 256 chunks (3 docs)
   - Finance: 129 chunks (1 docs)

📋 Category Distribution:
   - Guide: 703 chunks (3 docs)
   - Law: 256 chunks (3 docs)


## Step 9: Performance Tuning and Optimization

### Analyze and Optimize Database Performance

In [ ]:
def performance_tuning():
    """Analyze and optimize database performance"""
    try:
        print("⚡ Analyzing database performance...")
        
        kb_manager = KnowledgeBaseManager()
        
        with kb_manager.get_connection() as conn:
            with conn.cursor() as cur:
                # Check table sizes
                cur.execute("""
                    SELECT 
                        schemaname,
                        tablename,
                        pg_size_pretty(pg_total_relation_size(schemaname||'.'||tablename)) as size
                    FROM pg_tables 
                    WHERE schemaname = 'public'
                    ORDER BY pg_total_relation_size(schemaname||'.'||tablename) DESC
                """)
                table_sizes = cur.fetchall()
                
                # Check index usage
                cur.execute("""
                    SELECT 
                        schemaname,
                        tablename,
                        indexname,
                        idx_scan,
                        idx_tup_read,
                        idx_tup_fetch
                    FROM pg_stat_user_indexes 
                    ORDER BY idx_scan DESC
                """)
                index_stats = cur.fetchall()
                
                # Analyze query performance
                print(f"📊 Performance Analysis:")
                
                print(f"\n💾 Table Sizes:")
                for schema, table, size in table_sizes:
                    print(f"   - {table}: {size}")
                
                print(f"\n🔍 Index Usage:")
                for schema, table, index, scans, reads, fetches in index_stats[:10]:
                    print(f"   - {index} ({table}): {scans:,} scans, {reads:,} reads")
                
                # Test different search scenarios
                print(f"\n🎯 Search Performance Tests:")
                
                # Test 1: Basic vector search
                test_embedding = np.random.random(VECTOR_DIMENSION).tolist()
                start_time = time.time()
                results = kb_manager.search_kb_similar_vectors(test_embedding, limit=20)
                basic_search_time = time.time() - start_time
                print(f"   - Basic vector search (20 results): {basic_search_time*1000:.1f}ms")
                
                # Test 2: Filtered search by entity
                start_time = time.time()
                results = kb_manager.search_kb_similar_vectors(
                    test_embedding, 
                    limit=20,
                    entity_filter=['DGE']
                )
                entity_search_time = time.time() - start_time
                print(f"   - Entity filtered search: {entity_search_time*1000:.1f}ms")
                
                # Test 3: Multi-filter search
                start_time = time.time()
                results = kb_manager.search_kb_similar_vectors(
                    test_embedding, 
                    limit=20,
                    entity_filter=['DGE'],
                    domain_filter=['HR'],
                    category_filter=['Guide']
                )
                multi_filter_time = time.time() - start_time
                print(f"   - Multi-filter search: {multi_filter_time*1000:.1f}ms")
                
                # Performance recommendations
                print(f"\n💡 Performance Recommendations:")
                
                if basic_search_time > 1.0:
                    print(f"   ⚠️ Basic search is slow (>{basic_search_time:.1f}s) - consider query optimization")
                else:
                    print(f"   ✅ Basic search performance is good ({basic_search_time*1000:.1f}ms)")
                
                if entity_search_time > basic_search_time * 1.5:
                    print(f"   ⚠️ Filtered search is significantly slower - check indexes")
                else:
                    print(f"   ✅ Filtered search performance is acceptable")
                
                # Database maintenance recommendations
                print(f"\n🔧 Maintenance Recommendations:")
                print(f"   - Run VACUUM ANALYZE regularly for optimal performance")
                print(f"   - Monitor index usage and remove unused indexes")
                print(f"   - Consider connection pooling for production workloads")
                print(f"   - Set up monitoring for query performance")
                
                return True
                
    except Exception as e:
        print(f"❌ Error during performance tuning: {e}")
        return False

# Performance tuning
tuning_success = performance_tuning()
print(f"⚡ Tuning Status: {'✅ Success' if tuning_success else '❌ Failed'}")

## Step 10: Final Summary and Next Steps

### Pipeline Completion Summary

In [ ]:
def final_summary():
    """Generate final pipeline summary"""
    try:
        print("📋 Production Ingestion Pipeline Summary")
        print("=" * 50)
        
        # Overall status
        steps = [
            ("Database Creation", db_success),
            ("Schema Creation", schema_success),
            ("YAML Mapping", mapping_success),
            ("Hierarchy Population", hierarchy_success),
            ("Cache Clearing", cache_success),
            ("Document Processing", processing_success),
            ("Database Verification", verification_success),
            ("Performance Tuning", tuning_success)
        ]
        
        successful_steps = sum(1 for _, success in steps if success)
        total_steps = len(steps)
        
        print(f"📊 Pipeline Status: {successful_steps}/{total_steps} steps completed")
        print(f"✅ Success Rate: {successful_steps/total_steps*100:.1f}%")
        
        print(f"\n📋 Step Details:")
        for step_name, success in steps:
            status = "✅ Success" if success else "❌ Failed"
            print(f"   - {step_name}: {status}")
        
        # Database summary
        if verification_success:
            kb_manager = KnowledgeBaseManager()
            with kb_manager.get_connection() as conn:
                with conn.cursor() as cur:
                    cur.execute("SELECT COUNT(*) FROM knowledge_base")
                    total_chunks = cur.fetchone()[0]
                    
                    cur.execute("SELECT COUNT(DISTINCT document_name) FROM knowledge_base")
                    unique_docs = cur.fetchone()[0]
                    
                    cur.execute("SELECT COUNT(*) FROM knowledge_base WHERE embedding IS NOT NULL")
                    embedding_count = cur.fetchone()[0]
                    
                    print(f"\n📊 Final Database State:")
                    print(f"   - Database: {PRODUCTION_DATABASE_NAME}")
                    print(f"   - Total chunks: {total_chunks:,}")
                    print(f"   - Unique documents: {unique_docs}")
                    print(f"   - Embeddings: {embedding_count:,}")
                    print(f"   - Vector dimension: {VECTOR_DIMENSION}")
                    print(f"   - Embedding model: {EMBEDDING_MODEL}")
        
        # Next steps
        print(f"\n🚀 Next Steps:")
        print(f"   1. Set up search API endpoints")
        print(f"   2. Implement BM25 hybrid search")
        print(f"   3. Create user interface for search")
        print(f"   4. Set up monitoring and alerts")
        print(f"   5. Deploy to production environment")
        
        # Configuration for next steps
        print(f"\n🔧 Configuration for Next Steps:")
        print(f"   - Database URL: postgresql://mustaqmollah@localhost:5432/{PRODUCTION_DATABASE_NAME}")
        print(f"   - Embedding Model: {EMBEDDING_MODEL}")
        print(f"   - Vector Dimension: {VECTOR_DIMENSION}")
        print(f"   - Remote Docling: {os.getenv('DOCLING_SERVER_URL')}")
        print(f"   - Allowed Languages: {ALLOWED_LANGUAGES}")
        
        # Final timestamp
        print(f"\n⏰ Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        if successful_steps == total_steps:
            print(f"\n🎉 Production ingestion pipeline completed successfully!")
            print(f"   The Abu Dhabi Government Knowledge Base is ready for production use.")
        else:
            print(f"\n⚠️  Pipeline completed with {total_steps - successful_steps} failed steps.")
            print(f"   Please review the failed steps and retry if necessary.")
        
        return True
        
    except Exception as e:
        print(f"❌ Error generating final summary: {e}")
        return False

# Generate final summary
final_summary()